Ce notebook permet de visualiser les résultats du réseau de neurones sous forme visuelle, sans calculer de statistiques mais de façon visible pour l'oeil humain.

In [1]:
#!/usr/bin/env python

import glob, os
import numpy as np
import cv2
import argparse
import matplotlib.pyplot as plt
import re
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def load_pfm(file):
    '''Inspired by MVSNet/mvsnet/preprocesss.py:load_pfm()'''
    
    color = None
    width = None
    height = None
    scale = None
    data_type = None
    header = file.readline().decode('UTF-8').rstrip()

    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')
    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('UTF-8'))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')
    # scale = float(file.readline().rstrip())
    scale = float((file.readline()).decode('UTF-8').rstrip())
    if scale < 0: # little-endian
        data_type = '<f'
    else:
        data_type = '>f' # big-endian
    data_string = file.read()
    data = np.frombuffer(data_string, data_type)
    shape = (height, width, 3) if color else (height, width)
    data = np.reshape(data, shape)
    data = cv2.flip(data, 0)
    return data

def imread(path):
    if path.endswith('npy'):
        image = np.load(path)
        image = np.squeeze(image)
    elif path.endswith('pfm'):
        image = load_pfm(open(path, 'rb'))
    elif path.endswith('dmb'):
        image = read_gipuma_dmb(path)
    elif path.endswith('jpg'):
        image = cv2.imread(path)
    else:
        image = cv2.imread(path)

    return image

In [2]:
import skimage.metrics
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipyfilechooser import FileChooser
from IPython.display import clear_output

# Create and display a FileChooser widget
#fc = FileChooser('/scratch_p/rafrantz/')
#fc.show_only_dirs = True

#def on_path_changed_fc(chooser):    
#    on_path_changed(chooser.selected_path)
    
#fc.register_callback(on_path_changed_fc)
#display(fc)

def compute_ssim_difference(images):
    ssim, diff_ssim = skimage.metrics.structural_similarity(images["truth"], images["inferred"], full=True)
    return diff_ssim


def on_path_changed(out_dir):    
    filenames = glob.glob(os.path.join(out_dir, 'depths_mvsnet', '*_init.pfm'))
    
    if not filenames:
        print("No files found in ", out_dir)

    # Iterate over all camera RGB images
    for path in filenames:
        
        print("=" * 100)
        
        dirname = os.path.join(os.path.dirname(path), os.pardir)
        basename = os.path.basename(path)
        index = os.path.splitext(basename)[0].split('_')[0]

        rgb_path = os.path.join(dirname, 'depths_mvsnet', index+'.jpg')
        inferred_depth_prob_path = os.path.join(dirname, 'depths_mvsnet', index+'_prob.pfm')
        inferred_depth_path = os.path.join(dirname, 'depths_mvsnet', index+'_init.pfm')
        real_depth_path = os.path.join(dirname, 'rendered_depth_maps', index+'.pfm')
        
        metadatas = [
            {
                'title': 'Camera',
                'id': 'rgb',
                'colorbar': True,
                'path': rgb_path
            },
            {
                'title': 'Confiance',
                'id': 'confiance',
                'colorbar': True,
                'path': inferred_depth_prob_path
            },
            {
                'title': 'Inferred depth',
                'id': 'inferred',
                'colorbar': True,
                'path': inferred_depth_path
            },
            {
                'title': 'Real depth (original size)',
                'id': 'truth_full',
                'colorbar': True,
                'path': real_depth_path
            },
            {
                'title': 'Real depth (downscaled)',
                'id': 'truth',
                'colorbar': True,
                'dynamic_image': lambda images: cv2.resize(images["truth_full"], dsize=images["inferred"].shape[::-1], interpolation=cv2.INTER_NEAREST)
            },
            {
                'title': '1 / Real depth (original size)',
                'id': '1/truth_full',
                'colorbar': True,
                'dynamic_image': lambda images: 1.0 / images['truth_full']
            },
            {
                'title': '1 / Real depth (downscaled)',
                'id': '1/truth',
                'colorbar': True,
                'dynamic_image': lambda images: 1.0 / images['truth']
            },
            {
                'title': 'SSIM difference',
                'id': 'ssim',
                'colorbar': True,
                'dynamic_image': compute_ssim_difference
            },
            {
                'title': '1 / Inferred depth',
                'id': '1/inferred',
                'colorbar': True,
                'dynamic_image': lambda images: 1.0 / images['inferred']
            },
        ]
        
        fig_cols = 2
        fig_rows = int(np.ceil(len(metadatas) / fig_cols))
        fig = plt.figure(figsize=(16, 4 * fig_rows))
        
        images = {}
        dimension = None
        
        # Load images from files
        for i, metadata in enumerate(metadatas):
            id = metadata["id"]
            
            if "path" in metadata:
                images[id] = imread(metadata["path"])

        # Load dynamic images
        for i, metadata in enumerate(metadatas):
            id = metadata["id"]
            
            if "dynamic_image" in metadata:        
                dynamic_image = metadata["dynamic_image"]
                images[id] = dynamic_image(images)
        
        
        # Render
        for i, metadata in enumerate(metadatas):
            title = metadata['title']
            id = metadata["id"]
            
            ax = fig.add_subplot(fig_rows, fig_cols, i + 1)
            ax.set_title(title)
            im = plt.imshow(images[id])
            
            if metadata["colorbar"]:
                divider = make_axes_locatable(ax)
                cax = divider.append_axes('right', size='10%', pad=0.05)
                fig.colorbar(im, cax=cax, orientation='vertical')
        
        fig.tight_layout()
        plt.show()


@interact(model='C')
def on_path_changed_interact(model):
    on_path_changed(f"/scratch_p/rafrantz/tf_model_revery_romeo_{model}/tests/hf6ag69kr0jdf8ycuw6pf8/")

interactive(children=(Text(value='C', description='model'), Output()), _dom_classes=('widget-interact',))